# Protecto Final Deep Learning
## Clasificación de texto
-------------------------------
**Integrante 1**: Ángela Patricia Ordoñez Castro...............................................**Código**:201628709

**Integrante 3**: Juan Sebastián Garcés Carranza.............................................**Código**: 201628949

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# Lectura de datos

In [48]:
file_list = drive.ListFile({'q': "'1oDWXiKj_RLgE-8e6JyAzdYaNogAoJQTg' in parents and trashed=false"}).GetList()
for file1 in file_list:
 print('title: %s, id: %s' % (file1['title'], file1['id']))

title: Clasificación texto proyecto final.ipynb, id: 1-E8TuGwcPipBm4dJ8QB8ITW_Tb2jUkxT
title: Clasificación Imagenes, id: 1gyfsAz2DiiRxDUcuHNzwMQSQUB7OFRgg
title: FinalProjectDL.ipynb, id: 1ym2u-c42vfLJoDpJ3uo654w4-CPwqZ5h
title: images_resize_gray.zip, id: 1qe15uaPoGKvhhgbVDRrRb3_HzILN9zQR
title: New.zip, id: 1VnsRoWh-AOSkk6mSNIJztO2tFgL9OWBG
title: dataTraining.csv, id: 1pITh54OuRysHL8YiPHGkRC-xMVIsjBSH
title: dataTesting.csv, id: 1gS_kSQpj0Yz-SbWVZ9pDcNMjd_AYkQ55
title: ExampleTextRF (2).ipynb, id: 12moYXAJeIop6vPyC78sYT9FDf5vBlX4G


In [0]:
train_downloaded = drive.CreateFile({'id': '1pITh54OuRysHL8YiPHGkRC-xMVIsjBSH'})
train_downloaded.GetContentFile('dataTraining.csv')
test_downloaded = drive.CreateFile({'id':'1gS_kSQpj0Yz-SbWVZ9pDcNMjd_AYkQ55'})
test_downloaded.GetContentFile('dataTesting.csv')

In [4]:
import pandas as pd
import numpy as np
dataTraining = pd.read_csv('dataTraining.csv')
dataTesting = pd.read_csv('dataTesting.csv')
dataTesting.head()

,Unnamed: 0,year,title,plot
0,1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
1,4,1978,Midnight Express,"the true story of billy hayes , an american c..."
2,5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
3,6,1950,Crisis,husband and wife americans dr . eugene and mr...
4,7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [5]:
dataTraining.head()

,Unnamed: 0,year,title,plot,genres,rating
0,3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
1,900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
2,6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
3,4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
4,2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [0]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

# Creamos Y 

Creamos una matriz de 1 y 0, donde 1 es si la película pertenece a una clase, y 0 en caso contrario

In [0]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

# Aplicación Machine Learning

Definimos un Random Forest para predecir la categoría de las  películas a partir de la sinopsis de cada una

In [0]:

clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=500, max_depth=10, random_state=50))

 Definimos las bases X y Y:
 

In [0]:
X=dataTraining['plot']
Y=le.fit_transform(dataTraining['genres'])


Separamos la muestra inicial Training en Train y Test:

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

Definimos un  CountVectorizer con ngram, max_features y stemmer



In [11]:
vect = CountVectorizer(ngram_range=(1, 2), max_features=1000)
X_dtm = vect.fit_transform(dataTraining['plot'])
print(vect.get_feature_names()[:50])

['able', 'able to', 'about', 'about the', 'about to', 'accident', 'across', 'act', 'action', 'actually', 'affair', 'after', 'after the', 'again', 'against', 'against the', 'age', 'agent', 'ago', 'alex', 'alive', 'all', 'all of', 'all the', 'alone', 'along', 'along the', 'along with', 'already', 'also', 'although', 'always', 'america', 'american', 'among', 'an', 'an old', 'and', 'and has', 'and he', 'and her', 'and his', 'and is', 'and she', 'and that', 'and the', 'and their', 'and then', 'and they', 'and when']


In [0]:

stemmer = SnowballStemmer('english')
words = list(vect.vocabulary_.keys())[:100]
def split_into_stemmer(text):
    text = text.lower()
    words = text.split()
    return [stemmer.stem(word) for word in words]

In [13]:
vect1 = CountVectorizer(ngram_range=(1, 2), max_features=15000,analyzer=split_into_stemmer)
X_dtm1_train= vect1.fit_transform(X_train)
X_dtm1_test=vect1.transform(X_test)
X_dtm1_train.shape

(5289, 15000)

Corremos el modelo Random Forest con el primer CounVectorizer definido:

In [14]:
clf.fit(X_dtm1_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0, warm_start=False),
          n_jobs=1)

Prediciendo con el Random Forest y el primer CountVectorizer , obtenemos el siguiente resutado:

In [15]:
y_pred1 = clf.predict_proba(X_dtm1_test)
roc_auc_score(y_test, y_pred1, average='macro')

0.8373680414701181

Intentando mejorar el resultado anterior, cambiamos la función CountVectirizer por TfidVectorizer:

In [16]:
vect2 = TfidfVectorizer(ngram_range=(1, 2),max_features=15000,analyzer=split_into_stemmer)
X_dtm2_train = vect2.fit_transform(X_train)
X_dtm2_test = vect2.transform(X_test)
X_dtm2_train.shape

(5289, 15000)

Estimamos un nuevo Random Forest:

In [17]:
Modelo2=clf.fit(X_dtm2_train, y_train)
y_pred2 = clf.predict_proba(X_dtm2_test)
roc_auc_score(y_test, y_pred2, average='macro')

0.83952160425667

# Aplicación Deep learning

In [0]:
#!pip install livelossplot

In [0]:
#Librerias a utilizar:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.datasets import reuters
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from keras.layers import Dropout, BatchNormalization
from sklearn.feature_extraction.text import CountVectorizer
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam, TFOptimizer
from livelossplot import PlotLossesKeras
%matplotlib inline
from keras import backend as K

Definimos la estructura de una red neuronal:

In [0]:
max_words=15000

Modelo3= Sequential()
Modelo3.add(Dense(300, input_shape=(max_words,)))
Modelo3.add(Activation('selu'))
Modelo3.add(BatchNormalization())
Modelo3.add(Dropout(0.7))
Modelo3.add(Dense(24, activation='sigmoid'))


In [61]:
#Compilamos el modelo usando la función de perdida "categorical_crossentropy" y el optimizador Adagrad:

Modelo3.compile(loss = 'categorical_crossentropy',
              optimizer = Adagrad(),
              metrics = ['accuracy'])

Modelo3.fit(X_dtm2_train, y_train, epochs=10, verbose=2)


Epoch 1/10
 - 4s - loss: 7.8202 - acc: 0.1989
Epoch 2/10
 - 2s - loss: 6.2288 - acc: 0.3207
Epoch 3/10
 - 2s - loss: 5.4580 - acc: 0.3742
Epoch 4/10
 - 2s - loss: 4.9540 - acc: 0.4029
Epoch 5/10
 - 2s - loss: 4.6576 - acc: 0.4158
Epoch 6/10
 - 2s - loss: 4.4152 - acc: 0.4203
Epoch 7/10
 - 2s - loss: 4.2554 - acc: 0.4303
Epoch 8/10
 - 2s - loss: 4.1130 - acc: 0.4396
Epoch 9/10
 - 2s - loss: 4.0169 - acc: 0.4511
Epoch 10/10
 - 2s - loss: 3.9280 - acc: 0.4413


Con la red neuronal entrenada, predecimos sobre el test:

In [36]:
y_pred3=Modelo3.predict_proba(X_dtm2_test)
roc_auc_score(y_test, y_pred3, average='macro')

0.8413707619076142

Con el fin de mejorar la red neuronal, cambiamos la función de activación de la primera capa "selu" por "relu":

In [0]:
max_words=15000

Modelo4= Sequential()
Modelo4.add(Dense(300, input_shape=(max_words,)))
Modelo4.add(Activation('relu'))
Modelo4.add(BatchNormalization())
Modelo4.add(Dropout(0.7))
Modelo4.add(Dense(24, activation='sigmoid'))


In [24]:
Modelo4.compile(loss = 'categorical_crossentropy',
              optimizer = Adagrad(),
              metrics = ['accuracy'])

Modelo4.fit(X_dtm2_train, y_train, epochs=10, verbose=2)

Epoch 1/10
 - 3s - loss: 7.0793 - acc: 0.2702
Epoch 2/10
 - 2s - loss: 5.7445 - acc: 0.3222
Epoch 3/10
 - 2s - loss: 5.2166 - acc: 0.3485
Epoch 4/10
 - 2s - loss: 4.8597 - acc: 0.3906
Epoch 5/10
 - 2s - loss: 4.6200 - acc: 0.4084
Epoch 6/10
 - 2s - loss: 4.4639 - acc: 0.4207
Epoch 7/10
 - 2s - loss: 4.3246 - acc: 0.4473
Epoch 8/10
 - 2s - loss: 4.2227 - acc: 0.4432
Epoch 9/10
 - 2s - loss: 4.1368 - acc: 0.4560
Epoch 10/10
 - 2s - loss: 4.0595 - acc: 0.4490


In [25]:
y_pred4=Modelo4.predict_proba(X_dtm2_test)
roc_auc_score(y_test, y_pred4, average='macro')

0.8750357159912382

Corremos un nevo modelo, agregando un Dropout de 0.3:

In [0]:
max_words=15000

Modelo5= Sequential()
Modelo5.add(Dense(300, input_shape=(max_words,)))
Modelo5.add(Activation('relu'))
Modelo5.add(Dropout(0.3))
Modelo5.add(BatchNormalization())
Modelo5.add(Dropout(0.7))
Modelo5.add(Dense(24, activation='sigmoid'))

In [31]:
Modelo5.compile(loss = 'categorical_crossentropy',
              optimizer = Adagrad(),
              metrics = ['accuracy'])

Modelo5.fit(X_dtm2_train, y_train, epochs=50, verbose=2)

Epoch 1/50
 - 3s - loss: 5.1512 - acc: 0.3984
Epoch 2/50
 - 2s - loss: 5.0784 - acc: 0.4120
Epoch 3/50
 - 2s - loss: 4.9916 - acc: 0.4110
Epoch 4/50
 - 2s - loss: 4.8519 - acc: 0.4207
Epoch 5/50
 - 2s - loss: 4.8297 - acc: 0.4279
Epoch 6/50
 - 2s - loss: 4.7535 - acc: 0.4337
Epoch 7/50
 - 2s - loss: 4.6846 - acc: 0.4358
Epoch 8/50
 - 2s - loss: 4.6538 - acc: 0.4432
Epoch 9/50
 - 2s - loss: 4.6016 - acc: 0.4422
Epoch 10/50
 - 2s - loss: 4.5351 - acc: 0.4422
Epoch 11/50
 - 2s - loss: 4.5418 - acc: 0.4472
Epoch 12/50
 - 3s - loss: 4.5252 - acc: 0.4464
Epoch 13/50
 - 2s - loss: 4.4382 - acc: 0.4451
Epoch 14/50
 - 3s - loss: 4.4568 - acc: 0.4470
Epoch 15/50
 - 2s - loss: 4.4216 - acc: 0.4411
Epoch 16/50
 - 3s - loss: 4.3802 - acc: 0.4593
Epoch 17/50
 - 2s - loss: 4.3612 - acc: 0.4487
Epoch 18/50
 - 2s - loss: 4.3361 - acc: 0.4506
Epoch 19/50
 - 3s - loss: 4.3397 - acc: 0.4441
Epoch 20/50
 - 3s - loss: 4.3129 - acc: 0.4475
Epoch 21/50
 - 2s - loss: 4.2917 - acc: 0.4419
Epoch 22/50
 - 2s - lo

Epoch 47/50
 - 2s - loss: 4.0539 - acc: 0.4540
Epoch 48/50
 - 3s - loss: 4.0310 - acc: 0.4538
Epoch 49/50
 - 2s - loss: 4.0465 - acc: 0.4494
Epoch 50/50
 - 3s - loss: 4.0158 - acc: 0.4494


In [32]:
y_pred5=Modelo5.predict_proba(X_dtm2_test)
roc_auc_score(y_test, y_pred5, average='macro')

0.8793282211375691

El mejor modelo encontrado para predecir el género de las películas fue el modelo 5

In [0]:
#Insumo para stacking:
pd.DataFrame(y_pred5, index=X_test.index, columns=cols).to_csv('Y_validation_RN5.csv', index_label='ID')

In [0]:
#Insumo para stacking:
pd.DataFrame(y_test, index=X_test.index, columns=cols).to_csv('Y_test.csv', index_label='ID')

In [0]:
X_test_dtm = vect2.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = Modelo5.predict_proba(X_test_dtm)

pd.DataFrame(y_pred_test_genres, index=dataTesting['Unnamed: 0'], columns=cols).to_csv('pred_genres_text_RN5.csv', index_label='ID')

In [0]:
from google.colab import files
files.download('Y_validation_RN5.csv')